# 0. Import

In [ ]:
! pip install datasets
! pip install transformers
! pip install sentencepiece

In [4]:
import pandas as pd
from datasets import *
from transformers import RobertaTokenizerFast, RobertaConfig, RobertaForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer
from tokenizers import *
import torch
import os
import json

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# 1. Clean Korean Legal Corpus (CKLC)

## 1-1. Law data

In [ ]:
df = pd.read_csv('./data/cases(20220521).csv', index_col=False)  # index_col = 'case_number
df = df.dropna(how='any')
print(df.isnull().sum())
print(">> Number of Korea legal precedents : ", len(df))

case_name            0
case_number          0
date                 0
case_code            0
judgment_issue       0
judgment_summary     0
judgment_contents    0
dtype: int64
>> Number of Korea legal precedents :  62919


In [ ]:
df.head()

,case_name,case_number,date,case_code,judgment_issue,judgment_summary,judgment_contents
0,미성년자의제강간,2021노824,20220415.0,400102.0,피고인이 카카오톡 오픈채팅방을 통해 미성년자 여 11세 을 알게 된 후 당시 13세...,피고인이 카카오톡 오픈채팅방을 통해 미성년자 여 11세 을 알게 된 후 당시 13세...,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
2,손해배상(기),2021나24173,20220323.0,400101.0,이 을 상대로 소송을 제기하자 이 과 소송위임계약을 체결하여 이 을 대리하여 소송을...,이 을 상대로 소송을 제기하자 이 과 소송위임계약을 체결하여 이 을 대리하여 소송을...,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
4,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,20220317.0,400101.0,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 특히 환자가 병원에서...,의사가 진찰 치료 등의 의료행위를 할 때에는 사람의 생명 신체 건강을 관리하는 업무...,"【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."
5,상표법위반·업무상배임[타인의 상표가 부착된 제품을 무상으로 제공한 경우 상표법위반죄...,2021도2180,20220317.0,400102.0,1 상표법상 상표의 사용 및 상품의 의미 2 피고인 은 상표권자의 허락 없이 상표를...,1 상표법상 상표의 사용이란 상품 또는 상품의 포장에 상표를 표시하는 행위 상품 또...,【피 고 인】 피고인 1 외 1인【상 고 인】 검사【원심판결】 서울서부지법 2021...
6,소유권이전등기·소유권이전등기[피고로부터 부동산을 증여받은 원고들이 피고를 상대로 증...,"2017다207475, 207482",20220311.0,400101.0,1 수증자의 범죄행위를 원인으로 한 증여계약의 해제를 규정하고 있는 민법 제556조...,1 민법 제556조 제1항 제1호는 수증자가 증여자에 대하여 증여자 또는 그 배우자...,"【원고(반소피고), 상고인】 원고(반소피고) (소송대리인 법무법인(유한) 세종 담당..."


## 1-2. Data Sum

In [ ]:
# Data_Sum
files = [
         'law_summary.txt',
         'law_issue.txt',
         'law_contents.txt',
         'contract.txt'
         ]
dataset = load_dataset("text", data_files=files, split="train")

data = dataset.train_test_split(test_size=0.08)

In [7]:
# Text Data check!
for t in data['train']['text'][:10]:
    print(t)
    print("===="*50)

채무불이행 사유 등 계약이행에 관한 현저한 위험이 발생할 경우 각 당사자는 위험을 발생시킨 상대방에 대해 본 계약에서 다른 정함이 없는 한 계약대금의 에 해당하는 계약보증금을 즉시 현금 금융기관이 발행한 자기앞 수표 포함 또는 금융기관이 발행한 지급보증서로 납부할 것을 요구할 수 있고 위 요구 시 요구받은 당사자는 이를 즉시 이행할 것을 확약한다
1 정보통신망을 이용한 불안감 조성행위가 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제74조 제1항 제3호 제44조의7 제1항 제3호 위반죄에 해당하기 위한 요건 2 피고인이 약 3개월 동안 7회에 걸쳐 에게 휴대전화로 욕설 등이 담긴 문자메시지를 보냄으로써 공포심이나 불안감을 유발하는 문자를 반복적으로 도달하게 하였다는 내용으로 기소된 사안에서 위 정보통신망 이용촉진 및 정보보호 등에 관한 법률 위반의 공소사실을 유죄로 인정한 원심판결에 사실오인 또는 법리오해의 위법이 있다고 한 사례
향정신성의약품관리법상의 추징의 성질과 이득을 취한 바 없는 범법자에 대한추징 여부 적극
"【원고,피상고인】 【피고,상고인】 【원심판결】 대전지법 2002. 11. 1. 선고 2001나12186 판결【주문】  원심판결을 파기하고, 사건을 대전지방법원 본원 합의부에 환송한다.【이유】상고이유(상고이유서 제출기간 경과 후에 제출된 상고이유보충서는 상고이유를 보충하는 범위 내에서)를 본다.  1. 원심판결 이유에 의하면, 원심은 갑 제1호증(대출거래약정서, 피고들은 위 대출거래약정서에 날인된 각 인영의 성립을 인정하면서도 그 인영이 소외 1의 아들인 소외 2에 의해 위조되었다고 주장하나, 이에 부합하는 을 제3 내지 7호증의 각 기재, 제1심 증인 소외 1, 당심 증인 소외 3의 각 증언은 믿을 수 없고 달리 이를 인정할 만한 증거가 없다.), 갑 제4호증의 각 기재, 제1심 증인 소외 4의 증언에 변론의 전취지를 종합하면, 소외 1은 1998. 7. 16. 원고로부터 2,500만 원을 이율은 연 16.5%, 지연이율 연 23%, 변제기 

# 2. Tokenizer(Make Vocab)
    - Byte-Pair Encoding(BPE) : Roberta
    - WordPiece : BERT, DistilBERT
    - SentencePiece : ALBERT, XLNet, T5

In [ ]:
def dataset_to_text(dataset, output_filename='data.txt'):
    with open(output_filename, "w") as f:
        for t in dataset['text']:
            print(t, file=f)

# Save train / text dataset -> txt 
dataset_to_text(data["train"], "train_3.txt")
dataset_to_text(data["test"], "test_3.txt")

In [8]:
# Parameters
special_tokens = ["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
files = ["train_3.txt", "test_3.txt"]
vocab_size = 30522
max_length = 512
truncate_longer_samples = False  # No cut!

In [ ]:
# Initialize the WordPiece tokenizer
tokenizer = ByteLevelBPETokenizer()

# Train the tokenizer
tokenizer.train(
    files=files,
    vocab_size=vocab_size,
    special_tokens=special_tokens)

# Enable truncation up to the maximum "512 tokens"
tokenizer.enable_truncation(max_length=max_length)

In [ ]:
model_path = "pretrained-KorLawRoberta_2"
# Make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

# Save the tokenizer in model_path
tokenizer.save_model(model_path)

In [ ]:
model_path = "pretrained-KorLawRoberta_2"
tokenizer = RobertaTokenizerFast.from_pretrained(model_path, max_len=max_length)

# 3. Tokenizing the Dataset

In [ ]:
def encode_with_truncation(examples):
    """Mapping function to tokenize the sentences passed with truncation"""
    return tokenizer(examples["text"],
                     truncation=True,
                     padding="max_length",
                     max_length=max_length,
                     return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)


# 1.Encoding : The encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

# 2. Tokenizing the Train & Test Dataset 
train_dataset = data['train'].map(encode, batched=True)
test_dataset = data['test'].map(encode, batched=True)

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as 
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

train_dataset, test_dataset

In [ ]:
# 3. Main data processing function that will concatenate all texts from our dataset and generate chunks of max_seq_length.

def group_texts(examples):
    # 1.Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # 2. Drop the small remainder
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # 3. Split by Chunk of Max_len
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

# 'batched=True' : This map processes 2,000 Texts together, so group_texts THROWS AWAY a remainder for each of those groups of 2,000 texts. 
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True, batch_size=2000,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts,  batched=True, batch_size=2000,
                                  desc=f"Grouping texts in chunks of {max_length}")

# 4. Loading the Model

In [12]:
# 1. initialize the model with the config
model_config = RobertaConfig(vocab_size=vocab_size,
                             max_position_embeddings=514,
                             num_attention_heads=12,
                             num_hidden_layers=6,
                             type_vocab_size=1,
                             )

model = RobertaForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-7000"))

# 5. Pre-Training(RoBERTa Dynamic MLM Task)

In [13]:
# MLM : Randomly Masking 20% of the tokens For the Dynamic Roberta MLM Task
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=True,
                                                mlm_probability=0.2)
training_args = TrainingArguments(
    output_dir=model_path,            # output directory to where save model checkpoint
    evaluation_strategy="steps",      # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,              # number of training epochs
    per_device_train_batch_size=16,   # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,    # accumulating the gradients before updating the weights
    per_device_eval_batch_size=32,    # evaluation batch size
    logging_steps=1000,               # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

In [14]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train the model(110,000 Steps)
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 148813
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 11620


Step,Training Loss,Validation Loss
1000,1.912200,1.791506


The following columns in the evaluation set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12909
  Batch size = 32
Saving model checkpoint to pretrained-KorLawRoberta_2/checkpoint-1000
Configuration saved in pretrained-KorLawRoberta_2/checkpoint-1000/config.json
Model weights saved in pretrained-KorLawRoberta_2/checkpoint-1000/pytorch_model.bin
